# Question 2: DC-GAN

In [ ]:
!pip install torch torchvision
!pip install imageio
!pip install matplotlib
%mkdir -p /content/Question2/
%cd /content/Question2/

/content/Question2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Dependencies

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from six.moves.urllib.request import urlretrieve
import tarfile
import imageio
from urllib.error import URLError
from urllib.error import HTTPError
import warnings
warnings.filterwarnings('ignore')

## Helper Functions



In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

                
def to_var(tensor, cuda=True):
    if cuda:
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

    
def to_data(x):
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()


def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def gan_checkpoint(iteration, G, D, opts):
    G_path = os.path.join(opts.checkpoint_dir, 'G.pkl')
    D_path = os.path.join(opts.checkpoint_dir, 'D.pkl')
    torch.save(G.state_dict(), G_path)
    torch.save(D.state_dict(), D_path)


def merge_images(sources, targets, opts):
    _, _, h, w = sources.shape
    row = int(np.sqrt(opts.batch_size))
    merged = np.zeros([3, row * h, row * w * 2])
    for (idx, s, t) in (zip(range(row ** 2), sources, targets, )):
        i = idx // row
        j = idx % row
        merged[:, i * h:(i + 1) * h, (j * 2) * h:(j * 2 + 1) * h] = s
        merged[:, i * h:(i + 1) * h, (j * 2 + 1) * h:(j * 2 + 2) * h] = t
    return merged.transpose(1, 2, 0)


def generate_gif(directory_path, keyword=None):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith(".png") and (keyword is None or keyword in filename):
            img_path = os.path.join(directory_path, filename)
            print("adding image {}".format(img_path))
            images.append(imageio.imread(img_path))

    if keyword:
        imageio.mimsave(
            os.path.join(directory_path, 'anim_{}.gif'.format(keyword)), images)
    else:
        imageio.mimsave(os.path.join(directory_path, 'anim.gif'), images)


def create_image_grid(array, ncols=None):
    num_images, channels, cell_h, cell_w = array.shape
    if not ncols:
        ncols = int(np.sqrt(num_images))
    nrows = int(np.math.floor(num_images / float(ncols)))
    result = np.zeros((cell_h * nrows, cell_w * ncols, channels), dtype=array.dtype)
    for i in range(0, nrows):
        for j in range(0, ncols):
            result[i * cell_h:(i + 1) * cell_h, j * cell_w:(j + 1) * cell_w, :] = array[i * ncols + j].transpose(1, 2,
                                                                                                                 0)

    if channels == 1:
        result = result.squeeze()
    return result


def gan_save_samples(G, fixed_noise, iteration, opts):
    generated_images = G(fixed_noise)
    generated_images = to_data(generated_images)

    grid = create_image_grid(generated_images)

    path = os.path.join(opts.sample_dir, 'sample-{:06d}.png'.format(iteration))
    imageio.imwrite(path, grid)
    print('Saved {}'.format(path))

## Data loader

In [ ]:
def get_emoji_loader(emoji_type, opts):
    transform = transforms.Compose([
                    transforms.Scale(opts.image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

    train_path = os.path.join('data/emojis', emoji_type)
    test_path = os.path.join('data/emojis', 'Test_{}'.format(emoji_type))

    train_dataset = datasets.ImageFolder(train_path, transform)
    test_dataset = datasets.ImageFolder(test_path, transform)

    train_dloader = DataLoader(dataset=train_dataset, batch_size=opts.batch_size, shuffle=True, num_workers=opts.num_workers)
    test_dloader = DataLoader(dataset=test_dataset, batch_size=opts.batch_size, shuffle=False, num_workers=opts.num_workers)

    return train_dloader, test_dloader

## Helper modules

In [ ]:
# Generates a PyTorch Tensor of uniform random noise.
def sample_noise(batch_size, dim):
    return to_var(torch.rand(batch_size, dim) * 2 - 1).unsqueeze(2).unsqueeze(3)
  

def upsampling(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, spectral_norm=False):
    layers = []
    if stride>1:
        layers.append(nn.Upsample(scale_factor=stride))
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                           kernel_size=kernel_size, stride=1, padding=padding, bias=False)
    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)


def conv(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, init_zero_weights=False, spectral_norm=False):
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
    if init_zero_weights:
        conv_layer.weight.data = torch.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.001
            
    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)
  

class ResnetBlock(nn.Module):
    def __init__(self, conv_dim):
        super(ResnetBlock, self).__init__()
        self.conv_layer = conv(in_channels=conv_dim, out_channels=conv_dim, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        out = x + self.conv_layer(x)
        return out

## DCGAN

## Spectral Norm class

In [ ]:
def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)


class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))

        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False

    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)

    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)

## GAN generator

### Deconvolution

In [ ]:
class DCGenerator_Deconvolution(nn.Module):
    def __init__(self, noise_size, conv_dim, spectral_norm=False):
        super(DCGenerator_Deconvolution, self).__init__()
        self.main = nn.Sequential(
            # Input is 1 x 64 x 64
            nn.Conv2d(1, 64, (4, 4), (2, 2), (1, 1), bias=True),
            nn.LeakyReLU(0.2, True),
            # State size. 64 x 32 x 32
            nn.Conv2d(64, 128, (4, 4), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),
            # State size. 128 x 16 x 16
            nn.Conv2d(128, 256, (4, 4), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),
            # State size. 256 x 8 x 8
            nn.Conv2d(256, 512, (4, 4), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, True),
            # State size. 512 x 4 x 4
            nn.Conv2d(512, 1, (4, 4), (1, 1), (0, 0), bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.main(x)
        out = torch.flatten(out, 1)

        return out


In [ ]:
class DCGenerator_Upsampling(nn.Module):
    def __init__(self, noise_size, conv_dim, spectral_norm=False):
        super(DCGenerator_Upsampling, self).__init__()

        self.conv_dim = conv_dim

        self.linear_bn = upsampling(in_channels=100, out_channels=self.conv_dim*4,
                                    kernel_size=3, stride=2, padding=2, spectral_norm=spectral_norm)
        self.upsampling1 = upsampling(in_channels=self.conv_dim*4, out_channels=self.conv_dim*2,
                                      kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.upsampling2 = upsampling(in_channels=self.conv_dim*2, out_channels=self.conv_dim,
                                      kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.upsampling3 = upsampling(in_channels=self.conv_dim, out_channels=3,
                                      kernel_size=5, stride=2, spectral_norm=spectral_norm)

    def forward(self, z):
        batch_size = z.size(0)

        out = F.relu(self.linear_bn(z)).view(-1, self.conv_dim*4, 4, 4)    # BS x 128 x 4 x 4
        out = F.relu(self.upsampling1(out))  # BS x 64 x 8 x 8
        out = F.relu(self.upsampling2(out))  # BS x 32 x 16 x 16
        out = F.tanh(self.upsampling3(out))  # BS x 3 x 32 x 32
        
        out_size = out.size()
        if out_size != torch.Size([batch_size, 3, 32, 32]):
            raise ValueError("expect {} x 3 x 32 x 32, but get {}".format(batch_size, out_size))
        return out


## GAN discriminator

In [ ]:
class DCDiscriminator(nn.Module):
    def __init__(self, conv_dim=64, spectral_norm=False):
        super(DCDiscriminator, self).__init__()

        self.conv1 = conv(in_channels=3, out_channels=conv_dim, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=5, stride=2, 
                          spectral_norm=spectral_norm)
        self.conv3 = conv(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=5, 
                          stride=2, spectral_norm=spectral_norm)
        self.conv4 = conv(in_channels=conv_dim*4, out_channels=1, kernel_size=5, stride=2, padding=1, 
                          batch_norm=False, spectral_norm=spectral_norm)

    def forward(self, x):
        batch_size = x.size(0)

        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))

        out = self.conv4(out).squeeze()
        out_size = out.size()
        if out_size != torch.Size([batch_size,]):
            raise ValueError("expect {} x 1, but get {}".format(batch_size, out_size))
        return out

In [ ]:

def print_models(G_XtoY, G_YtoX, D_X, D_Y):
    if G_YtoX:
        print("                 G_XtoY                ")
        print("---------------------------------------")
        print(G_XtoY)
        print("---------------------------------------")

        print("                 G_YtoX                ")
        print("---------------------------------------")
        print(G_YtoX)
        print("---------------------------------------")

        print("                  D_X                  ")
        print("---------------------------------------")
        print(D_X)
        print("---------------------------------------")

        print("                  D_Y                  ")
        print("---------------------------------------")
        print(D_Y)
        print("---------------------------------------")
    else:
        print("                 G                     ")
        print("---------------------------------------")
        print(G_XtoY)
        print("---------------------------------------")

        print("                  D                    ")
        print("---------------------------------------")
        print(D_X)
        print("---------------------------------------")


def create_model(opts):
    """Builds the generators and discriminators.
    """
    if opts.Y is None:
        ### DC-GAN with upsampling
        G = DCGenerator_Upsampling(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
        D = DCDiscriminator(conv_dim=opts.d_conv_dim, spectral_norm=opts.spectral_norm)

        print_models(G, None, D, None)

        if torch.cuda.is_available():
            G.cuda()
            D.cuda()
            print('Models moved to GPU.')
        return G, D
    else:
        ### DC-GAN with deconvolution
        G = DCGenerator_Deconvolution(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
        D = DCDiscriminator(conv_dim=opts.d_conv_dim, spectral_norm=opts.spectral_norm)

        print_models(G, None, D, None)

        if torch.cuda.is_available():
            G.cuda()
            D.cuda()
            print('Models moved to GPU.')
        return G, D

def train(opts):
    """Loads the data, creates checkpoint and sample directories, and starts the training loop.
    """

    # Create train and test dataloaders for images from the two domains X and Y
    dataloader_X, test_dataloader_X = get_emoji_loader(emoji_type=opts.X, opts=opts)
    if opts.Y:
        dataloader_Y, test_dataloader_Y = get_emoji_loader(emoji_type=opts.Y, opts=opts)

    # Create checkpoint and sample directories
    create_dir(opts.checkpoint_dir)
    create_dir(opts.sample_dir)

    # Start training
    if opts.Y is None:
        G, D = gan_training_loop(dataloader_X, test_dataloader_X, opts)
        return G, D

def print_opts(opts):
    """Prints the values of all command-line arguments.
    """
    print('=' * 80)
    print('Opts'.center(80))
    print('-' * 80)
    for key in opts.__dict__:
        if opts.__dict__[key]:
            print('{:>30}: {:<30}'.format(key, opts.__dict__[key]).center(80))
    print('=' * 80)


### GAN training loop

In [ ]:
def gan_training_loop(dataloader, test_dataloader, opts):
    # Create generators and discriminators
    G, D = create_model(opts)

    g_params = G.parameters()  # Get generator parameters
    d_params = D.parameters()  # Get discriminator parameters

    # Create optimizers for the generators and discriminators
    g_optimizer = optim.Adam(g_params, opts.lr, [opts.beta1, opts.beta2])
    d_optimizer = optim.Adam(d_params, opts.lr * 2., [opts.beta1, opts.beta2])

    train_iter = iter(dataloader)

    test_iter = iter(test_dataloader)

    # Get some fixed data from domains X and Y for sampling. These are images that are held
    # constant throughout training, that allow us to inspect the model's performance.
    fixed_noise = sample_noise(100, opts.noise_size)  # # 100 x noise_size x 1 x 1

    iter_per_epoch = len(train_iter)
    total_train_iters = opts.train_iters

    losses = {"iteration": [], "D_fake_loss": [], "D_real_loss": [], "G_loss": []}

    gp_weight = 10

    try:
        for iteration in range(1, opts.train_iters + 1):

            # Reset data_iter for each epoch
            if iteration % iter_per_epoch == 0:
                train_iter = iter(dataloader)

            real_images, real_labels = train_iter.next()
            real_images, real_labels = to_var(real_images), to_var(real_labels).long().squeeze()

            for d_i in range(opts.d_train_iters):
                d_optimizer.zero_grad()

                # 1. Compute the discriminator loss on real images
                D_real_loss = torch.mean((D(real_images) - 1)**2) 

                # 2. Sample noise
                noise = sample_noise(real_images.shape[0], opts.noise_size)

                # 3. Generate fake images from the noise
                fake_images = G(noise)
                
                # 4. Compute the discriminator loss on the fake images
                D_fake_loss = torch.mean(D(fake_images)**2)

                # ---- Gradient Penalty ----
                if opts.gradient_penalty:
                    alpha = torch.rand(real_images.shape[0], 1, 1, 1)
                    alpha = alpha.expand_as(real_images).cuda()
                    interp_images = Variable(alpha * real_images.data + alpha * fake_images.data, requires_grad=True).cuda()
                    D_interp_output = D(interp_images)

                    gradients = torch.autograd.grad(outputs=D_interp_output, inputs=interp_images,
                                                    grad_outputs=torch.ones(D_interp_output.size()).cuda(),
                                                    create_graph=True, retain_graph=True)[0]
                    gradients = gradients.view(real_images.shape[0], -1)
                    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

                    gp = gp_weight * gradients_norm.mean()
                else:
                    gp = 0.0

                # 5. Compute the total discriminator loss
                D_total_loss = D_real_loss + D_fake_loss

                D_total_loss.backward()
                d_optimizer.step()


            g_optimizer.zero_grad()

            # 1. Sample noise
            noise = sample_noise(real_images.shape[0], opts.noise_size)

            # 2. Generate fake images from the noise
            fake_images = G(noise)

            # 3. Compute the generator loss
            G_loss = torch.mean((D(fake_images)-1)**2)

            G_loss.backward()
            g_optimizer.step()

            # Print the log info
            if iteration % opts.log_step == 0:
                losses['iteration'].append(iteration)
                losses['D_real_loss'].append(D_real_loss.item())
                losses['D_fake_loss'].append(D_fake_loss.item())
                losses['G_loss'].append(G_loss.item())
                print('Iteration [{:4d}/{:4d}] | D_real_loss: {:6.4f} | D_fake_loss: {:6.4f} | G_loss: {:6.4f}'.format(
                    iteration, total_train_iters, D_real_loss.item(), D_fake_loss.item(), G_loss.item()))

            # Save the generated samples
            if iteration % opts.sample_every == 0:
                gan_save_samples(G, fixed_noise, iteration, opts)

            # Save the model parameters
            if iteration % opts.checkpoint_every == 0:
                gan_checkpoint(iteration, G, D, opts)

    except KeyboardInterrupt:
        print('Exiting early from training.')
        return G, D

    plt.figure()
    plt.plot(losses['iteration'], losses['D_real_loss'], label='D_real')
    plt.plot(losses['iteration'], losses['D_fake_loss'], label='D_fake')
    plt.plot(losses['iteration'], losses['G_loss'], label='G')
    plt.legend()
    plt.savefig(os.path.join(opts.sample_dir, 'losses.png'))
    plt.close()
    return G, D
  

# Training


## Download dataset

In [ ]:
!mkdir data/
!mkdir data/emojis/

mkdir: cannot create directory ‘data/’: File exists
mkdir: cannot create directory ‘data/emojis/’: File exists


In [ ]:
!unzip /content/drive/MyDrive/emojis.zip -d data/

# Part A: Implementing DC-GAN

In [ ]:
SEED = 11

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':64,
              'noise_size':100,
              'num_workers': 0,
              'train_iters':20000,
              'X':'Windows',  # options: 'Windows' / 'Apple'
              'Y': 'Deconvolution',
              # Optimize Options
              'lr':0.0003,
              'beta1':0.5,
              'beta2':0.999,
             
              'batch_size':32, 
              'checkpoint_dir': 'results/checkpoints_gan',
              'sample_dir': 'results/samples_gan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 1
}
args.update(args_dict)

print_opts(args)
G, D = train(args)

generate_gif("results/samples_gan")

# Part B: Gradient Penalty

In [ ]:
!mkdir result_B
!mkdir result_B/checkpoints_gan
!mkdir result_B/samples_gan

In [ ]:
SEED = 11

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':64,
              'noise_size':100,
              'num_workers': 0,
              'train_iters':20000,
              'X':'Windows',  # options: 'Windows' / 'Apple'
              'Y': 'Deconvolution',
              # Optimize Options
              'lr':0.00001,
              'beta1':0.4,
              'beta2':0.999,
             
              'batch_size':32, 
              'checkpoint_dir': 'result_B/checkpoints_gan',
              'sample_dir': 'result_B/samples_gan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000,
              'spectral_norm': True,
              'gradient_penalty': True,
              'd_train_iters': 1
}
args.update(args_dict)

print_opts(args)
G, D = train(args)

generate_gif("result_B/samples_gan")

# Part C:Trying different hyperparameters

In [ ]:
!mkdir result_C
!mkdir result_C/checkpoints_gan
!mkdir result_C/samples_gan

In [ ]:
SEED = 11

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':64,
              'noise_size':100,
              'num_workers': 0,
              'train_iters':20000,
              'X':'Windows',  # options: 'Windows' / 'Apple'
              'Y': 'Deconvolution',
              # Optimize Options
              'lr':0.0001,
              'beta1':0.6,
              'beta2':0.999,
              'batch_size':32, 
              'checkpoint_dir': 'result_B/checkpoints_gan',
              'sample_dir': 'result_B/samples_gan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000,
              'spectral_norm': True,
              'gradient_penalty': True,
              'd_train_iters': 1,
}
args.update(args_dict)

print_opts(args)
G, D = train(args)

generate_gif("result_B/samples_gan")

# Part D: Upsampling

In [ ]:
!mkdir result_D
!mkdir result_D/checkpoints_gan
!mkdir result_D/samples_gan

In [ ]:
SEED = 11

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':64,
              'noise_size':100,
              'num_workers': 0,
              'train_iters':20000,
              'X':'Windows',  # options: 'Windows' / 'Apple'
              'Y': None,
              # Optimize Options
              'lr':0.0003,
              'beta1':0.5,
              'beta2':0.999,
             
              'batch_size':32, 
              'checkpoint_dir': 'result_D/checkpoints_gan',
              'sample_dir': 'result_D/samples_gan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000,
              'spectral_norm': True,
              'gradient_penalty': True,
              'd_train_iters': 1
}
args.update(args_dict)

print_opts(args)
G, D = train(args)

generate_gif("result_D/samples_gan")

                                      Opts                                      
--------------------------------------------------------------------------------
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 64                                     
                             noise_size: 100                                    
                            train_iters: 20000                                  
                                      X: Windows                                
                                     lr: 0.0003                                 
                                  beta1: 0.5                                    
                                  beta2: 0.999                                  
                             batch_size: 32                                     
                         che

Iteration [ 200/20000] | D_real_loss: 0.1648 | D_fake_loss: 0.1237 | G_loss: 0.6326
Saved result_C/samples_gan/sample-000200.png
Exiting early from training.
adding image result_C/samples_gan/losses.png
adding image result_C/samples_gan/sample-000200.png
adding image result_C/samples_gan/sample-000400.png
adding image result_C/samples_gan/sample-000600.png
adding image result_C/samples_gan/sample-000800.png
adding image result_C/samples_gan/sample-001000.png
adding image result_C/samples_gan/sample-001200.png
adding image result_C/samples_gan/sample-001400.png
adding image result_C/samples_gan/sample-001600.png
adding image result_C/samples_gan/sample-001800.png
adding image result_C/samples_gan/sample-002000.png
adding image result_C/samples_gan/sample-002200.png
adding image result_C/samples_gan/sample-002400.png
adding image result_C/samples_gan/sample-002600.png
adding image result_C/samples_gan/sample-002800.png
adding image result_C/samples_gan/sample-003000.png
adding image resu